In [1]:
import os
import shutil
from PIL import Image
import random
import math

In [2]:
FROM_IMAGE_PATH = "preprocessedDataset/from/"
TO_IMAGE_PATH = "preprocessedDataset/to/" 
SPLIT_SIZE_IMAGE = 256  
DATASET_TRAIN_PERCENT = 0.8  
PREPARE_DATASET = False


In [3]:
if not os.path.isdir("AerialImageDataset"):
    !curl -k https://files.inria.fr/aerialimagelabeling/getAerial.sh | bash
    shutil.rmtree("AerialImageDataset/test")

In [4]:

def split_image(original_image_path, output_folder, split_size):
    original_image = Image.open(original_image_path)
    width, height = original_image.size

    width = width - (width % split_size)
    height = height - (height % split_size)

    image_number = 0
    name, extension = os.path.splitext(os.path.basename(original_image_path))

    for y in range(0, height, split_size):
        for x in range(0, width, split_size):
            box = (x, y, x + split_size, y + split_size)
            cropped_image = original_image.crop(box)
            cropped_image.save(f"{output_folder}/{name}_{image_number}{extension}")
            image_number += 1

In [5]:

def create_dataset(train_percent):
    
    files = [element for element in os.listdir(FROM_IMAGE_PATH) if not element.startswith(".") if ".tif" in element]
    for i in range(math.ceil(len(files) * train_percent)):
        element = random.choice([element for element in os.listdir(FROM_IMAGE_PATH) if not element.startswith(".") if ".tif" in element])
        shutil.move(FROM_IMAGE_PATH + element, FROM_IMAGE_PATH + "train/" + element )
        shutil.move(TO_IMAGE_PATH + element, TO_IMAGE_PATH + "train/" + element )

    for element in [element for element in os.listdir(FROM_IMAGE_PATH) if not element.startswith(".") if ".tif" in element]:
        shutil.move(FROM_IMAGE_PATH + element, FROM_IMAGE_PATH + "test/" + element )
        shutil.move(TO_IMAGE_PATH + element, TO_IMAGE_PATH + "test/" + element )

In [ ]:
if PREPARE_DATASET: 
    os.makedirs(FROM_IMAGE_PATH + "test")
    os.makedirs(FROM_IMAGE_PATH + "train")
    os.makedirs(TO_IMAGE_PATH + "test/")
    os.makedirs(TO_IMAGE_PATH + "train")

In [6]:
if PREPARE_DATASET: 
    gt_path = "AerialImageDataset/train/gt/"
    for img in os.listdir(gt_path):
        split_image(gt_path + img, FROM_IMAGE_PATH, SPLIT_SIZE_IMAGE)

In [7]:
if PREPARE_DATASET:
    gt_path = "AerialImageDataset/train/images/"
    for img in os.listdir(gt_path):
        split_image(gt_path + img, TO_IMAGE_PATH, SPLIT_SIZE_IMAGE)

In [8]:
if PREPARE_DATASET:    
    create_dataset(DATASET_TRAIN_PERCENT)